## Введение

Добро пожаловать в краткое руководство по использованию возможностей LangChain! Этот урок познакомит вас с различными концепциями и инструментами, которые помогут эффективнее разрабатывать проекты на базе искусственного интеллекта.

Мы рассмотрим такие темы, как:

- Установка библиотек.
- Получение API-ключей OpenAI.
- Генерация ответов с использованием языковой модели.
- Создание цепочек (chains).
- Добавление памяти в модели.
- Использование векторных баз данных.
- Практический пример работы с Deep Lake в качестве векторного хранилища.

Кроме того, мы изучим, как применять инструменты и агенты, такие как агент векторного хранилища. Благодаря лаконичным объяснениям и примерам кода, это руководство станет полезным ресурсом для новичков в LangChain или тех, кто хочет улучшить свои рабочие процессы в сфере ИИ.

Мы также рассмотрим использование различных инструментов, включая Google Search, и обсудим, как комбинировать их с подходящими агентами для достижения нужных результатов. Вы узнаете, как запускать и управлять этими агентами, которые выступают в роли координаторов, выбирая инструменты на основе полученных запросов.

## Установка и API-ключи

Для начала установите необходимые пакеты следующей командой:

In [ ]:
pip -q install langchain==0.0.208 deeplake==3.9.27 openai==0.27.8 tiktoken  

**Чтобы получить доступ к сервисам OpenAI:**

1. Если у вас ещё нет аккаунта, зарегистрируйтесь на https://platform.openai.com/. Если аккаунт уже есть, перейдите к шагу 5.
2. Заполните форму регистрации, указав имя, email и пароль.
3. OpenAI отправит вам письмо с подтверждением. Перейдите по ссылке в письме.
4. Подтвердите email и укажите номер телефона для верификации.
5. Авторизуйтесь на https://platform.openai.com/.
6. Перейдите в раздел API-ключей: https://platform.openai.com/account/api-keys.
7. Нажмите Create new secret key, задайте ключу понятное имя и сохраните его.

Теперь вы можете использовать возможности OpenAI в своих проектах

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

activeloop_token = user_secrets.get_secret("ACTIVELOOP_TOKEN")
openai_api_key = user_secrets.get_secret("OPENAI_API_KEY")

os.environ["ACTIVELOOP_TOKEN"] = activeloop_token
os.environ["OPENAI_API_KEY"] = openai_api_key

print("Activeloop token exists:", "ACTIVELOOP_TOKEN" in os.environ)
print("OpenAI key exists:", "OPENAI_API_KEY" in os.environ)

## Вызов языковой модели (LLM)

Основная функция LangChain заключается в вызове языковой модели (LLM) с конкретным запросом. Чтобы продемонстрировать это, рассмотрим пример сервиса, который предлагает персонализированные программы тренировок на основе целей и предпочтений пользователя.

Сначала импортируем обёртку для работы с LLM:

In [ ]:
from langchain.llms import OpenAI

**Параметр temperature**

Этот параметр управляет случайностью выходных данных моделей OpenAI:

- 0: вывод предсказуем, подходит для задач, требующих стабильности.
- 1.0: вывод случайный и креативный, но редко рекомендуется для практических задач.
- 0.70–0.90: баланс между надёжностью и креативностью, идеален для творческих задач.

Оптимальное значение подбирается экспериментально для каждого сценария. В примере ниже инициализируется модель GPT-3.5 Turbo:

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

**Пример использования**

Запросим у модели персонализированную тренировку:

In [ ]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))

## Цепочки (Chains) в LangChain

Цепочка объединяет несколько компонентов для решения типовых задач. Самый популярный тип — LLMChain, который включает:

- PromptTemplate (шаблон запроса)
- Модель (LLM или ChatModel)
- Опциональный парсер вывода

Как работает LLMChain:

1. Принимает входные переменные.
2. Форматирует их в запрос с помощью PromptTemplate.
3. Передаёт запрос модели.
4. При наличии парсера — преобразует вывод модели в нужный формат.

**Пример: генерация названия компании**

Создадим цепочку для генерации названия компании, производящей экологичные бутылки для воды:

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))

## Память (Memory) в LangChain

Память в LangChain — это механизм, который сохраняет и управляет историей диалога между пользователем и ИИ. 

Она помогает поддерживать контекст и согласованность взаимодействия, позволяя модели генерировать более релевантные и точные ответы. 
Например, ConversationBufferMemory выступает обёрткой вокруг ChatMessageHistory, извлекая сообщения и передавая их в цепочку для улучшения контекстной генерации.

**Пример использования памяти:**

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)

**Как это работает?**

- В разделе Current conversation видно, как память сохраняет историю.
- После каждого запроса пользователя диалог обновляется: добавляются реплики и ответы ИИ.
- При генерации нового ответа модель использует эту историю как контекст, что повышает согласованность и релевантность ответов.

Таким образом, память позволяет LangChain поддерживать длительные диалоги, учитывая предыдущие взаимодействия.

## Deep Lake VectorStore
Deep Lake — это хранилище для эмбеддингов и их метаданных в контексте LLM-приложений. Оно позволяет выполнять гибридный поиск по эмбеддингам и атрибутам для эффективного извлечения данных. Интеграция с LangChain упрощает разработку и развёртывание приложений.

Преимущества Deep Lake:

- Мультимодальность: поддерживает хранение текстов, изображений, аудио, видео и их векторных представлений.
- Серверless-архитектура: облачные датасеты можно создавать и управлять ими без настройки серверов.
- Совместимость с ML-фреймворками: датасеты можно конвертировать в DataLoader для обучения моделей в PyTorch или TensorFlow.

**Настройка API-токена Activeloop:**

1. Зарегистрируйтесь на Activeloop.
2. На главной странице нажмите Create API token.
3. Укажите название токена и срок действия.
4. Скопируйте токен и сохраните его в переменной окружения ACTIVELOOP_TOKEN:

In [ ]:
os.environ["ACTIVELOOP_TOKEN"] = activeloop_token

**⚠️ Важно:** Не храните токены в коде — это угроза безопасности. Используйте переменные окружения или защищённые конфиги.

In [ ]:
!pip -q install deeplake==3.9.27  

**Пример работы с Deep Lake:**

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings  
from langchain.vectorstores import DeepLake  
from langchain.text_splitter import RecursiveCharacterTextSplitter  
from langchain.llms import OpenAI  
from langchain.chains import RetrievalQA  

# Инициализация моделей  
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)  
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")  

# Подготовка документов  
texts = [  
    "Napoleon Bonaparte was born in 15 August 1769",  
    "Louis XIV was born in 5 September 1638"  
]  
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  
docs = text_splitter.create_documents(texts)  

# Создание датасета
my_activeloop_org_id = "xmarva"  
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"  
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"  

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)  
 
db.add_documents(docs)  

**Создание цепочки RetrievalQA:**

RetrievalQA — это цепочка, которая объединяет два ключевых компонента:

- Извлечение (Retrieval): Извлечение релевантных документов из векторной базы данных
- Генерация (QuestionAnswering): Использование языковой модели для формирования ответа на основе найденных документов.

chain_type="stuff" — контекст из найденных документов "вставляется" прямо в промпт LLM.

In [ ]:
retrieval_qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=db.as_retriever()  
)  

**Инициализация агента с инструментом RetrievalQA:**

Агент — это сущность, которая использует инструменты (tools) для выполнения задачи. RetrievalQA становится одним из таких инструментов.

- Создаётся инструмент на основе RetrievalQA:
- Указывается имя, описание, и функция (retrieval_qa.run).
- Агент анализирует запрос и решает, какой инструмент использовать.
- Результат работы инструмента передаётся в LLM для финального ответа.

Тип агента ZERO_SHOT_REACT_DESCRIPTION:

- Агент не имеет памяти о предыдущих шагах.
- Принимает решение на основе текущего запроса и описаний инструментов.
- Формат ответа: Мысль → Действие → Наблюдение → Ответ.

In [ ]:
from langchain.agents import initialize_agent, Tool  
from langchain.agents import AgentType  

tools = [  
    Tool(  
        name="Retrieval QA System",  
        func=retrieval_qa.run,  
        description="Useful for answering questions."  
    ),  
]  

agent = initialize_agent(  
    tools,  
    llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True  
)  

# Запрос к агенту  
response = agent.run("When was Napoleone born?")  
print(response)  

**Добавление новых данных в датасет**

Чтобы векторная база (Deep Lake) могла отвечать на новые вопросы, в неё нужно добавлять актуальную информацию.

После добавления данных пересоздайте цепочку RetrievalQA и агента, чтобы они учитывали обновлённые данные.
Эмбеддинги для новых документов генерируются автоматически при вызове add_documents().

In [ ]:
# Загрузка существующего датасета  
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)  

# Новые документы  
texts = [  
    "Lady Gaga was born in 28 March 1986",  
    "Michael Jeffrey Jordan was born in 17 February 1963"  
]  
docs = text_splitter.create_documents(texts)  

# Добавление в датасет  
db.add_documents(docs)  

# Повторная инициализация агента  
retrieval_qa = RetrievalQA.from_chain_type(  
    llm=llm, chain_type="stuff", retriever=db.as_retriever()  
)  

tools = [  
    Tool(  
        name="Retrieval QA System",  
        func=retrieval_qa.run,  
        description="Useful for answering questions."  
    ),  
]  

agent = initialize_agent(  
    tools,  
    llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True  
)  

# Тестирование  
response = agent.run("When was Michael Jordan born?")  
print(response)  

## Агенты в LangChain

В LangChain агенты — это высокоуровневые компоненты, которые используют языковые модели для определения последовательности действий. Действием может быть использование инструмента (например, поиск в Google) или возврат ответа пользователю. Инструменты — это функции для конкретных задач: поиск в интернете, запросы к базам данных, выполнение кода.

Агенты работают по циклу:

- Решение (LLM выбирает действие)
- Выполнение действия (использует инструмент)
- Анализ результата
- Повторение до завершения задачи

## Типы агентов в LangChain

- Zero-shot-react-description: Выбирает инструменты на основе их описаний (без предварительного обучения).
- React-docstore: Работает с документами через инструменты Search (поиск документа) и Lookup (поиск термина в документе).
- Self-ask-with-search: Использует инструмент Intermediate Answer для поиска фактов (аналог Google Search).
- Conversational-react-description: Для диалогов, использует память о предыдущих взаимодействиях.

## Пример: Агент с Google Search

1. Настройка переменных окружения. Установите GOOGLE_API_KEY и GOOGLE_CSE_ID для доступа к Google Search API. Инструкция здесь

In [ ]:
google_cse_id = user_secrets.get_secret("GOOGLE_CSE_ID")
openai_api_key = user_secrets.get_secret("GOOGLE_API_KEY")

os.environ["GOOGLE_CSE_ID"] = google_cse_id
os.environ["GOOGLE_API_KEY"] = openai_api_key

print("GOOGLE_CSE token exists:", "GOOGLE_CSE_ID" in os.environ)
print("GOOGLE_API exists:", "GOOGLE_API_KEY" in os.environ)

In [ ]:
%pip -q install --upgrade --quiet  langchain-google-community

In [ ]:
search = GoogleSearchAPIWrapper()  # Автоматически использует GOOGLE_API_KEY и GOOGLE_CSE_ID  

tools = [  
    Tool(  
        name="google-search",  
        func=search.run,  
        description="Search Google for recent results"  
    )  
]  

tools[0].run("Obama's first name?")

2. Импорт модулей

In [ ]:
from langchain.llms import OpenAI  
from langchain.agents import AgentType, load_tools, initialize_agent, Tool  
from langchain.utilities import GoogleSearchAPIWrapper  

3. Инициализация модели

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)  # temperature=0 для точных ответов  

4. Создание инструмента Google Search

In [ ]:
search = GoogleSearchAPIWrapper()  # Автоматически использует GOOGLE_API_KEY и GOOGLE_CSE_ID  

tools = [  
    Tool(  
        name="google-search",  
        func=search.run,  
        description="Поиск в Google для ответов на вопросы о текущих событиях"  
    )  
]  

5. Инициализация агента

In [ ]:
agent = initialize_agent(  
    tools,  
    llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Тип агента  
    verbose=True,  # Подробный вывод  
    max_iterations=6  # Лимит шагов для избежания бесконечных циклов  
)  

 6. Запрос к агенту

In [ ]:
response = agent("Какие последние новости о марсоходе?")  
print(response['output'])  

## Инструменты в LangChain

LangChain предоставляет различные инструменты для взаимодействия агентов с внешним миром. С их помощью можно создавать кастомных агентов для решения задач: поиск в интернете, ответы на вопросы, выполнение кода. В этом разделе рассмотрим типы инструментов и примеры их использования.

**Google Search и суммаризация текста**

В этом примере создаются два инструмента:

- Поиск в Google для получения актуальной информации.
- Суммаризация текста с использованием языковой модели.
- Импорт библиотек

In [ ]:
from langchain.llms import OpenAI  
from langchain.agents import Tool  
from langchain.utilities import GoogleSearchAPIWrapper  
from langchain.prompts import PromptTemplate  
from langchain.chains import LLMChain  
from langchain.agents import initialize_agent, AgentType  

Создание цепочки для суммаризации

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)  

# Шаблон для суммаризации  
prompt = PromptTemplate(  
    input_variables=["query"],  
    template="Напиши краткое содержание следующего текста: {query}"  
)  

summarize_chain = LLMChain(llm=llm, prompt=prompt)  

Определение инструментов

In [ ]:
# Убедитесь, что переменные окружения  
# GOOGLE_API_KEY и GOOGLE_CSE_ID установлены.  
search = GoogleSearchAPIWrapper()  

tools = [  
    Tool(  
        name="Search",  
        func=search.run,  
        description="поиск актуальной информации о событиях"  
    ),  
    Tool(  
        name="Summarizer",  
        func=summarize_chain.run,  
        description="суммаризация текстов"  
    )  
]  

Инициализация агента

In [ ]:
agent = initialize_agent(  
    tools,  
    llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Тип агента  
    verbose=True  # Подробный вывод  
)  

Запуск агента

In [ ]:
response = agent("Какие последние новости о марсоходе? Пожалуйста, сделай краткий обзор.")  
print(response['output'])  

Пример работы агента:

- Поиск новостей с помощью инструмента Search.
- Суммаризация результатов через инструмент Summarizer.

Другие инструменты LangChain

- SerpAPI: Интеграция с поисковой системой для получения данных.
- PythonREPLTool: Выполнение Python-кода внутри агента.
- Кастомные инструменты: Создание специализированных функций под свои задачи (см. документацию LangChain).


LangChain открывает широкие возможности для разработки интеллектуальных агентов. Освоив базовые концепции (инструменты, цепочки, память), вы сможете создавать сложные AI-приложения. Для углублённого изучения рекомендуем пройти полный курс по LangChain. Удачи в разработке! 🚀